### Content of this notebook

Two regression problems: predict number of crimes per week and average number of crimes per day 
Preparation code does both. Some steps are the same where otheres are different
weekly specific variables are designated with prefix _w and daily average with _d
Regression algoritms are the same and will be executed twice by changing input 

#### Import nesessary modules

In [20]:
import numpy as np
import math as math 
import pandas as pd
import matplotlib as plt
import seaborn as sns
from matplotlib import pyplot as plt
import datetime
import calendar
# from datetime import datetime

from astral.sun import sun
from astral import LocationInfo

import warnings

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

from shapely.geometry.point import Point
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

#### Read input data

In [21]:
df_all = pd.read_csv('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/NYPD_Complaint_All_Clean.csv', sep=",")

subway = pd.read_csv('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/SUBWAY_ENTRANCE.csv')

nynta = pd.read_csv ('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/nynta.csv')

nyc_pop = pd.read_csv('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv')
nyc_pop2010 = nyc_pop[nyc_pop['Year'] == 2010]

holidays = pd.read_csv ('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/US_Holiday_Dates.csv')

#### Retreive holiday data

In [22]:
#===== Retrieve holidays and use nubmer of hilidays per week
holidays['Date'] = pd.to_datetime(holidays['Date'], format='%Y-%m-%d', errors = 'coerce')
holidays['WEEK'] = holidays['Date'].dt.isocalendar().week 
holidays['YEAR'] = holidays['Date'].dt.year
holidays_per_week= holidays.groupby(['YEAR' , 'WEEK']).size().reset_index(name='H_COUNTS')

#### Subways per census tract

In [23]:
#===== Determine number of subway stations per census tract region
#===================================================================
nynta['geometry'] = nynta['the_geom'].apply(wkt.loads)
nynta_geo_df = gpd.GeoDataFrame(nynta, geometry='geometry')
# nynta_geo_df.info()
nynta_geo_df['geometry']

poly = nynta_geo_df['geometry'] 

#===== Retrieve subway entrances geo location and count # of stations per each census tract
subway['geometry'] = subway['the_geom'].apply(wkt.loads)
subway_geo_df = gpd.GeoDataFrame(subway, geometry='geometry')

def subwayST(x):
    point = Point(x['geometry'].x, x['geometry'].y)
    if sum(poly.contains(point)) == 1:
        code = nynta_geo_df[poly.contains(point)]['NTACode'].item()
    else:
        code = 'NA'
    return (code)
    
subway_geo_df['NTACode'] = subway_geo_df.apply(subwayST, axis=1) 

subway_per_tract =  subway_geo_df.groupby(['NTACode']).size().reset_index(name='SUB_CNT')

#### Define counts (group by) for two regression problems: Weekly totals and daily average 

In [24]:
#Define crime category for prediction
#---------------------------------------------
#++++++++++++++ Weeky total set up   +++++++++++++++++
# ofns_list = ['BURGLARY', 'INTOXICATED & IMPAIRED DRIVING', 'DANGEROUS DRUGS']
ofns_list = ['PETIT LARCENY', 'GRAND LARCENY']
# ofns_list = ['PETIT LARCENY', 'HARRASSMENT 2', 'ASSAULT 3 & RELATED OFFENSES', 
#               'CRIMINAL MISCHIEF & RELATED OF', 'OFF. AGNST PUB ORD SENSBLTY &', 'FELONY ASSAULT']
df_all_sel = df_all[df_all['OFNS_DESC'].isin(ofns_list) ]

#Define grouping for aggregation
df_count_w = df_all_sel.groupby(['NTACode', 'YEAR', 'WEEK']).size().reset_index(name='counts') #per day

#++++++++++++++ Daily average set up   +++++++++++++++++
#Define grouping for aggregation
df_count_d = df_all_sel.groupby(['NTACode', 'WEEK', 'DAY_OF_WEEK']).size().reset_index(name='counts') #per day
num_of_years = len(df_all_sel.YEAR.unique())
df_count_d.counts = df_count_d.counts/num_of_years

#### Add zero count records for the days or weeks where groupby resulted in no rows

In [25]:
all_nta_codes = pd.DataFrame(nynta.NTACode.unique())
all_nta_codes.columns = ['NTACode']

date_range = pd.date_range(start='1/1/2006', end='12/31/2020', freq='D')
date_range = pd.DatetimeIndex.to_frame(date_range).reset_index()

date_range['YEAR'] = date_range[0].dt.year
date_range['WEEK'] = date_range[0].dt.isocalendar().week
date_range['DAY_OF_WEEK'] = date_range[0].dt.dayofweek
date_range['ZERO_COUNT'] = 0

# date_range=date_range[['YEAR', 'WEEK','ZERO_COUNT']].sort_values(by=['YEAR', 'WEEK', 'ZERO_COUNT'], ascending=True) 

#++++++++++++++ Weeky total set up   +++++++++++++++++
date_range_w = date_range.groupby(['YEAR', 'WEEK', 'ZERO_COUNT']).size().reset_index()
date_range_w = date_range_w[['YEAR', 'WEEK', 'ZERO_COUNT']]

all_nta_codes['key'] = 0
date_range_w['key'] = 0
default_count_df_w  = all_nta_codes.merge(date_range_w, how='outer')
default_count_df_w = default_count_df_w.drop('key', axis=1)

default_count_df_w = default_count_df_w.merge(df_count_w,how='outer', on=['NTACode','YEAR', 'WEEK'], indicator=True)

default_count_df_w = default_count_df_w[default_count_df_w['_merge'] == 'left_only'] 
default_count_df_w = default_count_df_w[['NTACode','YEAR', 'WEEK', 'ZERO_COUNT']]

default_count_df_w = pd.concat([df_count_w, default_count_df_w], axis=0) 


default_count_df_w['counts'] = default_count_df_w['counts'].fillna(0)
default_count_df_w = default_count_df_w[['NTACode','YEAR', 'WEEK', 'counts']]

#++++++++++++++ Daily average set up   +++++++++++++++++
date_range_d = date_range.groupby(['WEEK', 'DAY_OF_WEEK', 'ZERO_COUNT']).size().reset_index()
date_range_d = date_range_d[['WEEK', 'DAY_OF_WEEK', 'ZERO_COUNT']]

all_nta_codes['key'] = 0
date_range_d['key'] = 0
default_count_df_d  = all_nta_codes.merge(date_range_d, how='outer')
default_count_df_d = default_count_df_d.drop('key', axis=1)

default_count_df_d = default_count_df_d.merge(df_count_d,how='outer', on=['NTACode','DAY_OF_WEEK', 'WEEK'], indicator=True)

default_count_df_d = default_count_df_d[default_count_df_d['_merge'] == 'left_only'] 
default_count_df_d = default_count_df_d[['NTACode','WEEK', 'DAY_OF_WEEK', 'ZERO_COUNT']]

default_count_df_d = pd.concat([df_count_d, default_count_df_d], axis=0) 

default_count_df_d['counts'] = default_count_df_d['counts'].fillna(0)
default_count_df_d = default_count_df_d[['NTACode','WEEK', 'DAY_OF_WEEK', 'counts']]


#### Merge data to sensus tract population data, subsway counts, holiday counts

In [26]:
# Merge data from complaints to population and subsway stations
#--------------------------------------------------------------
#++++++++++++++ Weeky total set up   +++++++++++++++++
df_count1_w  = default_count_df_w.merge(nyc_pop2010,left_on='NTACode',right_on='NTA Code').merge(nynta_geo_df,on='NTACode') \
    .merge(subway_per_tract,how="outer",on='NTACode') \
    .merge(holidays_per_week,how="left",left_on=('YEAR', 'WEEK'),right_on=('YEAR', 'WEEK'))
   
df_count1_w['SUB_CNT'] = df_count1_w['SUB_CNT'].fillna(0)   #if no subsway station found in tract - set to 0 intead of null
df_count1_w['H_COUNTS'] = df_count1_w['H_COUNTS'].fillna(0)   #if no holidays found for a week - set to 0 intead of null

# Retieve only needed columns
df_count2_w = df_count1_w[['NTACode', 'NTAName', 'YEAR', 'WEEK','BoroName', 'Shape_Leng', 'Population',
                            'Shape_Area', 'geometry','SUB_CNT', 'H_COUNTS', 'counts']]
                            
df_count2_w['POP_DENS'] = 1000* df_count2_w['Population']/df_count2_w['Shape_Area'] 
df_count2_w = df_count2_w[[ 'NTACode','POP_DENS', 'SUB_CNT', 'YEAR', 'WEEK', 'H_COUNTS', 'counts']]

#++++++++++++++ Daily average set up   +++++++++++++++++
df_count1_d = default_count_df_d.merge(nyc_pop2010,left_on='NTACode',right_on='NTA Code').merge(nynta_geo_df,on='NTACode') \
    .merge(subway_per_tract,how="outer",on='NTACode') 
 
df_count1_d['SUB_CNT'] = df_count1_d['SUB_CNT'].fillna(0)   #if no subsway station found in tract - set to 0 intead of null

# Retieve only needed columns
df_count2_d = df_count1_d[['NTACode', 'NTAName', 'WEEK', 'DAY_OF_WEEK', 'BoroName', 'Shape_Leng', 'Population',
                       'Shape_Area', 'geometry','SUB_CNT', 'counts']]
                            
df_count2_d['POP_DENS'] = 1000* df_count2_d['Population']/df_count2_d['Shape_Area'] 
df_count2_d = df_count2_d[[ 'NTACode','POP_DENS', 'SUB_CNT', 'WEEK', 'DAY_OF_WEEK', 'counts']]

#### Create dummy variables and separate predictors and target

In [27]:
#========================================================================
#== Create dummy variables, scale, and extract target variable  =========
#========================================================================
#++++++++++++++ Weeky total set up   +++++++++++++++++
df_count2_dummy_w = pd.get_dummies(df_count2_w, columns=[ 'NTACode', 'YEAR', 'WEEK'])
df_count2_dummy_w.POP_DENS = ( df_count2_dummy_w.POP_DENS - df_count2_dummy_w.POP_DENS.mean() ) / df_count2_dummy_w.POP_DENS.std()
df_count2_dummy_w.SUB_CNT = ( df_count2_dummy_w.SUB_CNT - df_count2_dummy_w.SUB_CNT.mean() ) / df_count2_dummy_w.SUB_CNT.std()
df_count2_dummy_w.H_COUNTS = ( df_count2_dummy_w.H_COUNTS - df_count2_dummy_w.H_COUNTS.mean() ) / df_count2_dummy_w.H_COUNTS.std()

X_w = np.asarray (df_count2_dummy_w.drop('counts', axis=1) )
y_w = np.asarray (df_count2_dummy_w['counts'])

#++++++++++++++ Daily average set up   +++++++++++++++++
df_count2_dummy_d = pd.get_dummies(df_count2_d, columns=[ 'NTACode', 'WEEK', 'DAY_OF_WEEK'])

df_count2_dummy_d.POP_DENS = ( df_count2_dummy_d.POP_DENS - df_count2_dummy_d.POP_DENS.mean() ) / df_count2_dummy_d.POP_DENS.std()
df_count2_dummy_d.SUB_CNT = ( df_count2_dummy_d.SUB_CNT - df_count2_dummy_d.SUB_CNT.mean() ) / df_count2_dummy_d.SUB_CNT.std()

X_d = np.asarray (df_count2_dummy_d.drop('counts', axis=1) )
y_d = np.asarray (df_count2_dummy_d['counts'])

#### Idenify features correlated to the target for weekly prediction

In [38]:
# Correlations in one-hot encoded dataframe
print(df_count2_dummy_w.corr()['counts'].abs().sort_values(ascending=False).head(20))

counts          1.000000
SUB_CNT         0.778283
NTACode_MN17    0.626360
NTACode_MN13    0.315631
POP_DENS        0.236549
YEAR_2020       0.211772
NTACode_MN23    0.177180
NTACode_MN24    0.168279
NTACode_MN12    0.132604
NTACode_MN40    0.107831
NTACode_BK82    0.103225
NTACode_BK38    0.090702
NTACode_QN29    0.089787
NTACode_MN15    0.089285
NTACode_BK61    0.087792
NTACode_MN25    0.076311
NTACode_QN22    0.063368
NTACode_MN14    0.061253
NTACode_MN34    0.059944
NTACode_MN11    0.059486
Name: counts, dtype: float64


#### Idenify features correlated to the target for daily average prediction

In [39]:
# Correlations in one-hot encoded dataframe
print(df_count2_dummy_d.corr()['counts'].abs().sort_values(ascending=False).head(20))

counts          1.000000
SUB_CNT         0.834118
NTACode_MN17    0.671297
NTACode_MN13    0.338276
POP_DENS        0.253520
NTACode_MN23    0.189892
NTACode_MN24    0.180352
NTACode_MN12    0.142117
NTACode_MN40    0.115567
NTACode_BK82    0.110630
WEEK_53         0.101409
NTACode_BK38    0.097209
NTACode_QN29    0.096228
NTACode_MN15    0.095690
NTACode_BK61    0.094090
NTACode_MN25    0.081785
NTACode_QN22    0.067915
NTACode_MN14    0.065647
NTACode_MN34    0.064244
NTACode_MN11    0.063754
Name: counts, dtype: float64


#### Assign X and y to run one of two problems (weekly total or daily avg) at a time

In [28]:
#++++++++++++++ Predictions using Weeky total set up   +++++++++++++++++
# X = X_w
# y = y_w

#++++++++++++++ Predictions using Daily average set up   +++++++++++++++++
X = X_d
y = y_d

# Regression algorithms start here

### KNN

#### Training and prediction of this model (as well as others) is extremply slow! For naive base run and hyperparameter tunning we will use a subset of data. We'll use a set reasonablly large subset to make a judgment while improving the total execution time.

#### First naive run 

In [29]:
X1 = X[:30000,:]
y1= y[:30000]
# X1 = X
# y1= y 

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

knn = KNeighborsRegressor(n_neighbors=50)
knn.fit(X_train, y_train)

print('\nAccuracy results for KNN Regression with train data')
print('======================================================')

y_predicted = knn.predict(X_train)

print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_train, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_train)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_train) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_train, y_predicted))

baseline = np.median(y_train)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_train)))
print("Baseline RMSE: %.3f "% np.sqrt(np.mean((baseline - y_train) ** 2)))


print('\nAccuracy results for KNN Regression with test data')
print('======================================================')

y_predicted = knn.predict(X_test)

# Explained variance score: 1 is perfect prediction
print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_test, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_test)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_test) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_test, y_predicted))
baseline = np.median(y_test)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_test)))
print("Baseline RMSE: %.3f"% np.sqrt(np.mean((baseline - y_test) ** 2)))


Accuracy results for KNN Regression with train data

Coefficient of determination r^2 variance score : 0.815
MAE: 0.321
RMSE: 0.442
MSE: 0.195
Baseline MAE: 0.757
Baseline RMSE: 1.055 

Accuracy results for KNN Regression with test data

Coefficient of determination r^2 variance score : 0.798
MAE: 0.336
RMSE: 0.465
MSE: 0.216
Baseline MAE: 0.763
Baseline RMSE: 1.063


#### Hyperparameter tunning

In [30]:
X1 = X[:30000,:]
y1= y[:30000]
# X1 = X
# y1= y 

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

neighbors = [3,5,10,20,50]
wghts=['uniform', 'distance']
distance=[1,2] #1-manhattan; 2-euclidean

models = {}
for i in range(len(neighbors)):       
    for j in range(len(wghts)):       
        for w in range(len(distance)): 
            knn = KNeighborsRegressor(n_neighbors=neighbors[i], weights=wghts[j],p=distance[w])
            knn.fit(X_train, y_train) 
            
            y_predicted = knn.predict(X_train)
            train_r2 = r2_score(y_train, y_predicted)
            
            y_predicted = knn.predict(X_test)
            test_r2 = r2_score(y_test, y_predicted) 
            
            key = [neighbors[i], wghts[j], distance[w]]
            models[tuple(key)] = [train_r2, test_r2]
            
models_sorted=sorted(models.items(), key=lambda elem: elem[1][1], reverse=True)  
     
for i in range(len(models_sorted)):
    print('Parameters: ', models_sorted[i][0],'; Trainig R2 score: {:.3f}; Test R2 score: {:.3f}'.format(models_sorted[i][1][0],models_sorted[i][1][1]) )

Parameters:  (50, 'distance', 2) ; Trainig R2 score: 1.000; Test R2 score: 0.799
Parameters:  (50, 'distance', 1) ; Trainig R2 score: 1.000; Test R2 score: 0.799
Parameters:  (50, 'uniform', 2) ; Trainig R2 score: 0.815; Test R2 score: 0.798
Parameters:  (50, 'uniform', 1) ; Trainig R2 score: 0.815; Test R2 score: 0.798
Parameters:  (20, 'uniform', 1) ; Trainig R2 score: 0.820; Test R2 score: 0.792
Parameters:  (20, 'uniform', 2) ; Trainig R2 score: 0.820; Test R2 score: 0.792
Parameters:  (20, 'distance', 1) ; Trainig R2 score: 1.000; Test R2 score: 0.792
Parameters:  (20, 'distance', 2) ; Trainig R2 score: 1.000; Test R2 score: 0.792
Parameters:  (10, 'uniform', 1) ; Trainig R2 score: 0.829; Test R2 score: 0.781
Parameters:  (10, 'uniform', 2) ; Trainig R2 score: 0.829; Test R2 score: 0.781
Parameters:  (10, 'distance', 1) ; Trainig R2 score: 1.000; Test R2 score: 0.781
Parameters:  (10, 'distance', 2) ; Trainig R2 score: 1.000; Test R2 score: 0.781
Parameters:  (5, 'uniform', 1) ; T

#### Use best model for prediction

In [31]:
#Use all data with best model from previous step
X1 = X 
y1 = y 

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

#++++++++++++++ Predictions best model for weekly  +++++++++++++++++
# knn = KNeighborsRegressor(n_neighbors=20, weights='distance', p=1)

#++++++++++++++ Predictions best model for daily average +++++++++++++++++
knn = KNeighborsRegressor(n_neighbors=50, weights='distance', p=1)

knn.fit(X_train, y_train)

print('\nAccuracy results for KNN Regression with train data')
print('======================================================')

y_predicted = knn.predict(X_train)

print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_train, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_train)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_train) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_train, y_predicted))

baseline = np.median(y_train)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_train)))
print("Baseline RMSE: %.3f "% np.sqrt(np.mean((baseline - y_train) ** 2)))


print('\nAccuracy results for KNN Regression with test data')
print('======================================================')

y_predicted = knn.predict(X_test)

# Explained variance score: 1 is perfect prediction
print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_test, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_test)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_test) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_test, y_predicted))
baseline = np.median(y_test)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_test)))
print("Baseline RMSE: %.3f"% np.sqrt(np.mean((baseline - y_test) ** 2)))


Accuracy results for KNN Regression with train data

Coefficient of determination r^2 variance score : 1.000
MAE: 0.000
RMSE: 0.000
MSE: 0.000
Baseline MAE: 1.050
Baseline RMSE: 2.082 

Accuracy results for KNN Regression with test data

Coefficient of determination r^2 variance score : 0.938
MAE: 0.326
RMSE: 0.513
MSE: 0.263
Baseline MAE: 1.077
Baseline RMSE: 2.121


## Linear Regression  

#### Hyperparameter tunning

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

models = {}

LRmodel = LinearRegression()
LRmodel.fit(X_train, y_train)       
            
y_predicted = LRmodel.predict(X_train)
train_r2 = r2_score(y_train, y_predicted)           

y_predicted = LRmodel.predict(X_test)
test_r2 = r2_score(y_test, y_predicted)
           
key = ['LR', 1]
models[tuple(key)] = [train_r2, test_r2]

alp = [.1,.3, .6, .9]

for i in range(len(alp)): 
    
    LRmodel = linear_model.Lasso(alpha=alp[i])
    LRmodel.fit(X_train, y_train)       
            
    y_predicted = LRmodel.predict(X_train)
    train_r2 = r2_score(y_train, y_predicted)           
    
    y_predicted = LRmodel.predict(X_test)
    test_r2 = r2_score(y_test, y_predicted)           
    key = ['Lasso', alp[i]]
    models[tuple(key)] = [train_r2, test_r2]
    
    LRmodel = Ridge(alpha=alp[i])
    LRmodel.fit(X_train, y_train)       
            
    y_predicted = LRmodel.predict(X_train)
    train_r2 = r2_score(y_train, y_predicted)           
    
    y_predicted = LRmodel.predict(X_test)
    test_r2 = r2_score(y_test, y_predicted)           
    key = ['Ridge', alp[i]]
    models[tuple(key)] = [train_r2, test_r2]
            
models_sorted=sorted(models.items(), key=lambda elem: elem[1][1], reverse=True)  
     
for i in range(len(models_sorted)):
    print('Parameters: ', models_sorted[i][0],'; Trainig R2 score {:.3f}; test R2 score: {:.3f}'.format(models_sorted[i][1][0],models_sorted[i][1][1]) )

Parameters:  ('Ridge', 0.9) ; Trainig R2 score 0.932; test R2 score: 0.935
Parameters:  ('Ridge', 0.6) ; Trainig R2 score 0.932; test R2 score: 0.935
Parameters:  ('Ridge', 0.3) ; Trainig R2 score 0.932; test R2 score: 0.935
Parameters:  ('Ridge', 0.1) ; Trainig R2 score 0.932; test R2 score: 0.935
Parameters:  ('LR', 1) ; Trainig R2 score 0.932; test R2 score: 0.935
Parameters:  ('Lasso', 0.1) ; Trainig R2 score 0.697; test R2 score: 0.707
Parameters:  ('Lasso', 0.3) ; Trainig R2 score 0.672; test R2 score: 0.682
Parameters:  ('Lasso', 0.6) ; Trainig R2 score 0.605; test R2 score: 0.616
Parameters:  ('Lasso', 0.9) ; Trainig R2 score 0.493; test R2 score: 0.503


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

LRmodel = LinearRegression()

LRmodel.fit(X_train, y_train)

print('\nAccuracy results for Linear Regression with train data')
print('======================================================')

y_predicted = LRmodel.predict(X_train)

print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_train, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_train)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_train) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_train, y_predicted))

baseline = np.median(y_train)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_train)))
print("Baseline RMSE: %.3f "% np.sqrt(np.mean((baseline - y_train) ** 2)))


print('\nAccuracy results for Linear Regression with test data')
print('======================================================')

y_predicted = LRmodel.predict(X_test)

# Explained variance score: 1 is perfect prediction
print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_test, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_test)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_test) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_test, y_predicted))
baseline = np.median(y_test)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_test)))
print("Baseline RMSE: %.3f"% np.sqrt(np.mean((baseline - y_test) ** 2)))


Accuracy results for Linear Regression with train data

Coefficient of determination r^2 variance score : 0.932
MAE: 0.326
RMSE: 0.529
MSE: 0.279
Baseline MAE: 1.050
Baseline RMSE: 2.082 

Accuracy results for Linear Regression with test data

Coefficient of determination r^2 variance score : 0.935
MAE: 0.331
RMSE: 0.526
MSE: 0.277
Baseline MAE: 1.077
Baseline RMSE: 2.121


## Random Forest Regression

In [34]:
X1 = X[:20000,:]
y1= y[:20000]
# X1 = X
# y1= y 

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

n_trees = [10,20,50,100, 200]

models = {}
for i in range(len(n_trees)):       
    regressor = RandomForestRegressor(n_estimators = n_trees[i], random_state = 0)
    regressor.fit(X_train, y_train)  
            
    y_predicted = regressor.predict(X_train)
    r2_train = r2_score(y_train, y_predicted) 
            
    y_predicted = regressor.predict(X_test)
    r2_test = r2_score(y_test, y_predicted)  
    
    key = [n_trees[i]]
    models[tuple(key)] = [r2_train, r2_test]
            
models_sorted=sorted(models.items(), key=lambda elem: elem[1][1], reverse=True)  
     
for i in range(len(models_sorted)):
    print('Parameters: ', models_sorted[i][0],'; Trainig R2 score: {:.3f}; Test R2 score: {:.3f}'.format(models_sorted[i][1][0],models_sorted[i][1][1]) )

Parameters:  (100,) ; Trainig R2 score: 0.981; Test R2 score: 0.859
Parameters:  (200,) ; Trainig R2 score: 0.982; Test R2 score: 0.859
Parameters:  (50,) ; Trainig R2 score: 0.981; Test R2 score: 0.858
Parameters:  (20,) ; Trainig R2 score: 0.979; Test R2 score: 0.855
Parameters:  (10,) ; Trainig R2 score: 0.976; Test R2 score: 0.853


#### Use best model for prediction

In [35]:
#Use best model from previous step
X1 = X
y1= y 

#++++++++++++++ Predictions best model for weekly  +++++++++++++++++
# best_n_trees = 200

#++++++++++++++ Predictions best model for daily average +++++++++++++++++
best_n_trees = 100

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

# create regressor object and fit with train data
regressor = RandomForestRegressor(n_estimators = best_n_trees, random_state = 0)
regressor.fit(X_train, y_train)  

print('\nAccuracy results for Random Forest Regression with train data')
print('======================================================')

y_predicted = regressor.predict(X_train)

print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_train, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_train)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_train) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_train, y_predicted))

baseline = np.median(y_train)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_train)))
print("Baseline RMSE: %.3f "% np.sqrt(np.mean((baseline - y_train) ** 2)))


print('\nAccuracy results for Random Forest Regression with test data')
print('======================================================')

y_predicted = regressor.predict(X_test)

# Explained variance score: 1 is perfect prediction
print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_test, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_test)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_test) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_test, y_predicted))
baseline = np.median(y_test)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_test)))
print("Baseline RMSE: %.3f"% np.sqrt(np.mean((baseline - y_test) ** 2)))


Accuracy results for Rendom Forest Regression with train data

Coefficient of determination r^2 variance score : 0.994
MAE: 0.110
RMSE: 0.161
MSE: 0.026
Baseline MAE: 1.050
Baseline RMSE: 2.082 

Accuracy results for Rendom Forest Regression with test data

Coefficient of determination r^2 variance score : 0.957
MAE: 0.297
RMSE: 0.429
MSE: 0.184
Baseline MAE: 1.077
Baseline RMSE: 2.121


## Multilayer perceptron (MLP) Regression

In [36]:
warnings.filterwarnings('ignore')

X1 = X[:20000,:]
y1= y[:20000]

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

# p_hidden_layer_neurons = [3,5,10,20,30]
# p_activation= ['logistic','tanh','relu']
# p_alpha=[0.1, .01, .001] 
# p_learning_rate=['constant', 'adaptive']
# p_solver=['lbfgs','sgd']
# p_learning_rate_init=[0.01, .001, .0001]
# p_max_iter=[200, 500, 1000]

p_hidden_layer_neurons = [10,20, 50]
p_activation= ['logistic', 'tanh','relu']
p_alpha=[.001] 
p_learning_rate=['adaptive']
p_solver=['sgd']
p_learning_rate_init=[0.001]
p_max_iter=[200, 500]

models = {}
for i in range(len(p_hidden_layer_neurons)):       
    for j in range(len(p_activation)):       
        for w in range(len(p_alpha)): 
            for u in range(len(p_solver)): 
                for z in range(len(p_learning_rate_init)): 
                    for v in range(len(p_max_iter)): 
                        for k in range(len(p_learning_rate)):
                            mlpregr = MLPRegressor(random_state=42, hidden_layer_sizes=(p_hidden_layer_neurons[i], ), alpha=p_alpha[w], solver=p_solver[u],   
                                max_iter=p_max_iter[v], learning_rate=p_learning_rate[k], activation=p_activation[j], learning_rate_init=p_learning_rate_init[z],
                                verbose=False)
 
                            mlpregr.fit(X_train, y_train)
                        
                            y_predicted = mlpregr.predict(X_train)
                            r2_train = r2_score(y_train, y_predicted) 
                        
                            y_predicted = mlpregr.predict(X_test)
                            r2_test = r2_score(y_test, y_predicted) 
                        
                            key = [p_hidden_layer_neurons[i], p_activation[j], p_alpha[w], p_solver[u],p_learning_rate_init[z],p_max_iter[v],p_learning_rate[k]]
                            models[tuple(key)] = [r2_train, r2_test]
            
models_sorted=sorted(models.items(), key=lambda elem: elem[1][1], reverse=True)  
     
for i in range(len(models_sorted)):
    print('Parameters: ', models_sorted[i][0],'; Trainig R2 score: {:.3f}; Test R2 score: {:.3f}'.format(models_sorted[i][1][0],models_sorted[i][1][1]) )


Parameters:  (20, 'relu', 0.001, 'sgd', 0.001, 200, 'adaptive') ; Trainig R2 score: 0.870; Test R2 score: 0.861
Parameters:  (20, 'relu', 0.001, 'sgd', 0.001, 500, 'adaptive') ; Trainig R2 score: 0.870; Test R2 score: 0.861
Parameters:  (50, 'relu', 0.001, 'sgd', 0.001, 200, 'adaptive') ; Trainig R2 score: 0.870; Test R2 score: 0.860
Parameters:  (50, 'relu', 0.001, 'sgd', 0.001, 500, 'adaptive') ; Trainig R2 score: 0.870; Test R2 score: 0.860
Parameters:  (10, 'relu', 0.001, 'sgd', 0.001, 200, 'adaptive') ; Trainig R2 score: 0.867; Test R2 score: 0.859
Parameters:  (10, 'relu', 0.001, 'sgd', 0.001, 500, 'adaptive') ; Trainig R2 score: 0.867; Test R2 score: 0.859
Parameters:  (10, 'logistic', 0.001, 'sgd', 0.001, 500, 'adaptive') ; Trainig R2 score: 0.859; Test R2 score: 0.848
Parameters:  (10, 'logistic', 0.001, 'sgd', 0.001, 200, 'adaptive') ; Trainig R2 score: 0.859; Test R2 score: 0.848
Parameters:  (20, 'logistic', 0.001, 'sgd', 0.001, 500, 'adaptive') ; Trainig R2 score: 0.858; T

#### Use Best Model

In [37]:
X1 = X 
y1 = y 

#Set up with best parameters chosen from previous step
s_alpha=.001
s_learning_rate='adaptive'
s_solver='sgd'
s_learning_rate_init=0.001

#++++++++++++++ Predictions best model for weekly  +++++++++++++++++
# s_hidden_layer_neurons = 50
# s_activation= 'logistic' 
s_max_iter=500

#++++++++++++++ Predictions best model for daily average +++++++++++++++++
s_hidden_layer_neurons = 20
s_activation= 'relu' 
s_max_iter=200

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

MLPmodel = MLPRegressor(random_state=42, hidden_layer_sizes=(s_hidden_layer_neurons,), alpha=s_alpha, solver=s_solver,   
        max_iter=s_max_iter, learning_rate=s_learning_rate, activation=s_activation, learning_rate_init=s_learning_rate_init,verbose=False)
 
MLPmodel.fit(X_train, y_train)

print('\nAccuracy results for MLP with train data')
print('======================================================')

y_predicted = MLPmodel.predict(X_train)

print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_train, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_train)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_train) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_train, y_predicted))

baseline = np.median(y_train)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_train)))
print("Baseline RMSE: %.3f "% np.sqrt(np.mean((baseline - y_train) ** 2)))


print('\nAccuracy results for MLP with test data')
print('======================================================')

y_predicted = MLPmodel.predict(X_test)

# Explained variance score: 1 is perfect prediction
print('\nCoefficient of determination r^2 variance score : %.3f' % r2_score(y_test, y_predicted))
print("MAE: %.3f"% np.mean(abs(y_predicted - y_test)))
print("RMSE: %.3f"% np.sqrt(np.mean((y_predicted - y_test) ** 2)))
print("MSE: %.3f" % mean_squared_error(y_test, y_predicted))
baseline = np.median(y_test)
print("Baseline MAE: %.3f"% np.mean(abs(baseline - y_test)))
print("Baseline RMSE: %.3f"% np.sqrt(np.mean((baseline - y_test) ** 2)))


Accuracy results for MLP with train data

Coefficient of determination r^2 variance score : 0.957
MAE: 0.287
RMSE: 0.419
MSE: 0.175
Baseline MAE: 1.050
Baseline RMSE: 2.082 

Accuracy results for MLP with test data

Coefficient of determination r^2 variance score : 0.959
MAE: 0.293
RMSE: 0.416
MSE: 0.173
Baseline MAE: 1.077
Baseline RMSE: 2.121
